# Imports

In [2]:
import numpy as np
import xarray as xr
from scipy import constants, integrate, optimize
%matplotlib qt5
import matplotlib.pyplot as plt

# Model without drifts

##  Equations

The starting equations are the 1D poloidal particle and momentum transport equations:

$$
\begin{align}
\frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ n \sin(\theta) v_\parallel \right] &= S_p =
\begin{cases}
S_{p,\mathrm{core}}, &\ \left| y \right| < y_\mathrm{x-point} \\
-S_{p,\mathrm{PFR}}, &\ \left| y \right| \geq y_\mathrm{x-point}
\end{cases} \\
\frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ n \left( v_\parallel^2 + c_s^2 \right) \right] &= 0
\end{align}
$$

Here $y$ represents a poloidal-angle-like variable, so $y = 0$ half-way between targets, $y = \pm y_\mathrm{x-point}$ at the lower/upper X-points, and $y = \pm y_\mathrm{divertor}$ at the lower/upper divertors; $n$ is the plasma density, where it is assumed that there are no impurities and the ion species is hydrogenic, so $n \equiv n_i = n_e$; $\theta$ is the field-line pitch angle, which is assumed to be constant throughout SOL; $v_\parallel$ is the parallel plasma velocity; $S_p$ is the particle source density, which is assumed to be a positive constant throughout the portion of the SOL in contact with the core plasma and a negative constant throughout the portion of the SOL in contact with the private flux region; $c_s = \sqrt{2 k_B T / m_i}$ is the ion sound speed, where it is assumed that the temperature $T \equiv T_i = T_e$ is constant throughout the SOL.

The general solution to the momentum transport equation is

$$
n \left( v_\parallel^2 + c_s^2 \right) = B,
$$

where $B$ is a constant. The general solution to the particle transport equation changes throughout the SOL due to the discontinuous nature of $S_p$. Breaking the domain into three regions: a) where $y < -y_\mathrm{x-point}$, b) where $y_\mathrm{x-point} < y < -y_\mathrm{x-point}$, and c) where $y > y_\mathrm{x-point}$; the general solution is

$$
n v_\parallel =
\begin{cases}
-\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y + C_a, &\ y < -y_\mathrm{x-point} \\
\frac{S_{p,\mathrm{core}}}{\sin(\theta)} y + C_b, &\ \left| y \right| < y_\mathrm{x-point} \\
-\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y + C_c, &\ y > y_\mathrm{x-point}
\end{cases}
$$

where $C_a$, $C_b$, and $C_c$ are constants for each region. There are thus four unknowns: $B$, $C_a$, $C_b$, and $C_c$. Requiring that $n$ and $v_\parallel$ are continuous at the boundaries between regions gives two equations, and the Bohm criterion at the lower/upper divertors gives two more equations, allowing the full system to be solved. At the lower divertor, the Bohm criterion is $v_\parallel(-y_\mathrm{divertor}) = -c_s$, and at the upper divertor it is $v_\parallel(y_\mathrm{divertor}) = c_s$. Plugging these into the momentum transport equation gives

$$
n(-y_\mathrm{divertor}) = n(y_\mathrm{divertor}) = \frac{B}{2 c_s^2}.
$$

Thus, the density is the same at the upper and lower divertors. The total integrated particle source into the SOL must be equal to the particle flux to the divertors, which is given by $\Gamma_\mathrm{divertor} = \left[ n(-y_\mathrm{divertor}) + n(y_\mathrm{divertor}) \right] c_s \sin(\theta)$. The total integrated particle source is $S_p^\mathrm{total} \equiv \int S_p \mathop{}\!\mathrm{d} y = 2 y_\mathrm{x-point} S_{p,\mathrm{core}} - 2 \left( y_\mathrm{divertor} - y_\mathrm{x-point} \right) S_{p,\mathrm{PFR}}$. Note that this model requires $S_p^\mathrm{total} \geq 0$, i.e., the total particle flux into the PFR cannot exceed the total particle flux from the core into the SOL. Setting $S_p^\mathrm{total} = \Gamma_\mathrm{divertor}$ gives

$$
S_p^\mathrm{total} = \frac{B \sin(\theta)}{c_s} \rightarrow B = \frac{c_s S_p^\mathrm{total}}{\sin(\theta)}.
$$

So

$$
n_\mathrm{divertor} = \frac{S_p^\mathrm{total}}{2 c_s \sin(\theta)}.
$$

Plugging the Bohm criterion into the particle transport equations for regions a and c gives

$$
\begin{align}
-n_\mathrm{divertor} c_s &= \frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y_\mathrm{divertor} + C_a, \\
n_\mathrm{divertor} c_s &= -\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y_\mathrm{divertor} + C_c.
\end{align}
$$

Yielding

$$
\begin{align}
C_a &= -n_\mathrm{divertor} c_s - \frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y_\mathrm{divertor}, \\
C_c &= n_\mathrm{divertor} c_s + \frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} y_\mathrm{divertor}.
\end{align}
$$

Now we apply the interface condition that $n$ and $v_\parallel$ are continuous at the boundaries between regions. At the boundary between regions a and b this yields

$$
\begin{align}
n(-y_\mathrm{x-point}) \, v_\parallel(-y_\mathrm{x-point}) &= -\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} (y_\mathrm{divertor} - y_\mathrm{x-point}) - n_\mathrm{divertor} c_s, \\
n(-y_\mathrm{x-point}) \, v_\parallel(-y_\mathrm{x-point}) &= -\frac{S_{p,\mathrm{core}}}{\sin(\theta)} y_\mathrm{x-point} + C_b.
\end{align}
$$

Subtracting these equations yields

$$
\begin{align}
C_b &= \frac{S_{p,\mathrm{core}}}{\sin(\theta)} y_\mathrm{x-point} -\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} (y_\mathrm{divertor} - y_\mathrm{x-point}) - n_\mathrm{divertor} c_s, \\
&= \frac{1}{2 \sin(\theta)} \left[ 2 S_{p,\mathrm{core}} y_\mathrm{x-point} - 2 S_{p,\mathrm{PFR}} (y_\mathrm{divertor} - y_\mathrm{x-point}) \right] - n_\mathrm{divertor} c_s, \\
&= \frac{S_p^\mathrm{total}}{2 \sin(\theta)} - n_\mathrm{divertor} c_s, \\
&= 0.
\end{align}
$$

One could also conclude that $C_b = 0$ from the up/down symmetry of the system.

The final model equations are then

$$
\begin{align}
n \left( v_\parallel^2 + c_s^2 \right) &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)}, \\
n v_\parallel &=
\begin{cases}
-\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} (y + y_\mathrm{divertor}) - n_\mathrm{divertor} c_s, &\ y < -y_\mathrm{x-point}, \\
\frac{S_{p,\mathrm{core}}}{\sin(\theta)} y, &\ \left| y \right| < y_\mathrm{x-point}, \\
-\frac{S_{p,\mathrm{PFR}}}{\sin(\theta)} (y - y_\mathrm{divertor}) + n_\mathrm{divertor} c_s, &\ y > y_\mathrm{x-point},
\end{cases} \\
\end{align}
$$

where

$$
\begin{align}
S_p^\mathrm{total} &= 2 y_\mathrm{x-point} S_{p,\mathrm{core}} - 2 \left( y_\mathrm{divertor} - y_\mathrm{x-point} \right) S_{p,\mathrm{PFR}}, \\
n_\mathrm{divertor} &= \frac{S_p^\mathrm{total}}{2 c_s \sin(\theta)}.
\end{align}
$$

These implicit equations are nasty to solve explicitly for $n$ and $v_\parallel$, so below they are solved numerically. In this model $v_\parallel = 0$ at $y = 0$, which is also the location of maximum density, $n_\mathrm{max} = 2 n_\mathrm{divertor}$.

## Explicit calculation

In [3]:
y = np.linspace(-1, 1, num=201) # poloidal angle variable runs from -1 (lower divertor) to +1 (upper divertor)
y_xpoint = 0.75 # relative distance of x-points from upstream position to divertors
field_line_pitch_angle = 0.001 # radians, this is not actually needed for this calculation since I treat the downstream_density as an input parameter
ion_mass = constants.proton_mass # kilograms
temperature = 50 # electron-volts
downstream_density = 2e18 # particles per cubic meter, I use this as an input parameter instead of core particle source since I have more intuition on what numbers are reasonable

ion_sound_speed = np.sqrt(2*temperature * constants.e / ion_mass)
y_norm = y / y_xpoint
parallel_velocity = np.zeros_like(y)
density = np.zeros_like(y)

# solution in region below lower X-point
below_xpoint = y_norm <= -1
parallel_velocity[below_xpoint] = -ion_sound_speed
density[below_xpoint] = downstream_density

# solution in region above upper X-point
above_xpoint = y_norm >= 1
parallel_velocity[above_xpoint] = ion_sound_speed
density[above_xpoint] = downstream_density

# solution in region between X-points
between_xpoints = np.logical_and(y_norm > -1, y_norm < 1)
root = np.where(y_norm[between_xpoints] > 0, -1, 1) # parallel velocity calculation requires different roots on either side of y=0
with np.errstate(divide='ignore'): # ignore runtime warnings caused by division by zero at y=0
    parallel_velocity[between_xpoints] = ion_sound_speed * ((1 / y_norm[between_xpoints]) + root * np.sqrt((1 / (y_norm[between_xpoints]**2)) - 1))
    density[between_xpoints] = downstream_density * y_norm[between_xpoints] * ion_sound_speed / parallel_velocity[between_xpoints]

# solution at y=0 needs to be hard-coded since general expression is indeterminate at y=0
parallel_velocity[y_norm == 0] = 0
density[y_norm == 0] = 2 * downstream_density

## Implicit calculation

In [8]:
# Inputs
y = np.linspace(-1, 1, num=201) # poloidal angle variable runs from -1 (lower divertor) to +1 (upper divertor)
y_xpoint = 0.75 # relative distance of x-points from upstream position to divertors
field_line_pitch_angle = 0.001 # radians
ion_mass = constants.proton_mass # kilograms
temperature = 50 # electron-volts
upstream_density = 1e19 # particles per cubic meter, I use this as an input parameter instead of core particle source since I have more intuition on what numbers are reasonable
particle_loss_fraction = 0 # this number quantifies how many of the particles sourced from the core are lost to the PFR before reaching the divertor, it must be equal to 0 because model breaks down otherwise
ion_sound_speed = np.sqrt(2*temperature * constants.e / ion_mass)

# Equation definitions
def transport_equations(density, parallel_velocity, y, ion_sound_speed, field_line_pitch_angle, upstream_density, particle_loss_fraction):
    """Note: in these equations density is normalized by the upstream density and parallel velocity is normalized by the ion sound speed to improve numerical stability of the solver"""
    core_particle_source = upstream_density * ion_sound_speed * np.sin(field_line_pitch_angle) / (2 * y_xpoint * (1 - particle_loss_fraction))
    pfr_particle_sink = particle_loss_fraction * core_particle_source * y_xpoint / (1 - y_xpoint)
    if y < -y_xpoint:
        particle_transport_equation = density * parallel_velocity + pfr_particle_sink * (y + 1) / (np.sin(field_line_pitch_angle) * upstream_density * ion_sound_speed) + 1 / 2
    elif y > y_xpoint:
        particle_transport_equation = density * parallel_velocity + pfr_particle_sink * (y - 1) / (np.sin(field_line_pitch_angle) * upstream_density * ion_sound_speed) - 1 / 2
    else: # between X-points
        particle_transport_equation = density * parallel_velocity - core_particle_source * y / (np.sin(field_line_pitch_angle) * upstream_density * ion_sound_speed)
    momentum_transport_equation = density * (parallel_velocity**2 + 1) - 1
    return [
        particle_transport_equation,
        momentum_transport_equation,
    ]

# Numerically solve the system of implicit equations individually at each position
parallel_velocity = np.zeros_like(y)
density = np.zeros_like(y)
solution_found = np.zeros_like(y)
solver_message = []
for i in range(len(y)):
    (density[i], parallel_velocity[i]), _, solution_found[i], mesg  = optimize.fsolve(
        func=lambda x : transport_equations(x[0], x[1], y[i], ion_sound_speed, field_line_pitch_angle, upstream_density, particle_loss_fraction),
        x0=[1, -1],
        full_output=True,
    )
    solver_message.append(mesg)
density[solution_found != 1] = np.nan
parallel_velocity[solution_found != 1] = np.nan

# Un-normalize the output
density = density * upstream_density
parallel_velocity = parallel_velocity * ion_sound_speed

## Plot density and parallel flow profiles

In [9]:
fig, ax1 = plt.subplots(constrained_layout=True)
ax2 = ax1.twinx()
ax1.set_xticks([-1, -y_xpoint, 0, y_xpoint, 1])
ax1.set_xticklabels(['lower divertor', 'lower X-point', '0', 'upper X-point', 'upper divertor'], minor=False, rotation=45)
ax1.set_xlabel('y')
ax1.set_title('Profiles in absence of drifts')

ax1.plot(y, parallel_velocity, color='tab:blue')
# ax1.axhline(0, linestyle='--')
ax1.set_yticks([-ion_sound_speed, 0, ion_sound_speed])
ax1.set_yticklabels(['$-c_s$', '0', '$c_s$'], minor=False)
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_ylabel(r'Parallel velocity', color='tab:blue')

ax2.plot(y, density, color='tab:red')
ax2.set_ylim(bottom=0)
ax2.set_yticks([0, upstream_density / 2, upstream_density])
ax2.set_yticklabels(['0', '$n_d$', '$2 n_d$'], minor=False)
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.set_ylabel('Density', color='tab:red');

print(max(density))
np.save("Kriete_density", density)
np.save("Kriete_parallel_velocity", parallel_velocity)
np.save("Kriete_y", y)

9.999999999999711e+18


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


# Model with drifts

## Starting equations

The starting equations are the 1D poloidal particle and momentum transport equations:

$$
\begin{align}
\frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ n \sin(\theta) v_\parallel + n \cos(\theta) v_\perp \right] &= S_p =
\begin{cases}
S_{p,\mathrm{core}}, &\ \left| y \right| < y_\mathrm{x-point} \\
S_{p,\mathrm{PFR}}, &\ \left| y \right| \geq y_\mathrm{x-point}
\end{cases} \\
\frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ n \left( v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right) \right] &= 0
\end{align}
$$

Here $y$ represents a poloidal-angle-like variable, so $y = 0$ half-way between targets, $y = \pm y_\mathrm{x-point}$ at the lower/upper X-points, and $y = \pm y_\mathrm{divertor}$ at the lower/upper divertors; $n$ is the plasma density, where it is assumed that there are no impurities and the ion species is hydrogenic, so $n \equiv n_i = n_e$; $\theta$ is the field-line pitch angle, which is assumed to be constant throughout SOL; $v_\parallel$ is the parallel plasma velocity; $v_\perp$ is the binormal $E \times B$ drift velocity, which is assumed to be constant throughout the SOL; $S_p$ is the particle source density; $c_s = \sqrt{2 k_B T / m_i}$ is the ion sound speed, where it is assumed that the temperature $T \equiv T_i = T_e$ is constant throughout the SOL (because the model applies only to the sheath-limited regime).

$S_p$ is assumed to be a positive constant throughout the portion of the SOL in contact with the core plasma. Ideally it would also have a negative term accounting for transport into the PFR throughout the portion of the SOL between the X-point and divertor. However, my attempts so far to have a nonzero particle "sink" into the PFR has resulted in equations that have no solution throughout parts of the SOL due to $v_\parallel$ going supersonic. The form of the PFR particle sink is given by Y. Feng et al. PPCF 1998:
$$
S_{p,\mathrm{PFR}} = -D \frac{n}{\Delta_r \lambda_n},
$$
where $D$ is the diffusion coefficient, $\Delta_r$ is the island radial width (on the order of 10 cm for W7-X), $\lambda_n$ is the density gradient scale length. In the sheath-limited regime, $\lambda_n$ can be estimated using
$$
\lambda_n = \sqrt{\frac{2 D L_c}{c_s}},
$$
where $L_c$ is the connection length.

## Solving the equations (assuming zero particle "sink" from transport into PFR)

The general solution to the momentum transport equation is

$$
n \left( v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right) = B,
$$

where $B$ is a constant. The general solution to the particle transport equation changes throughout the SOL due to the discontinuous nature of $S_p$. Breaking the domain into three regions: a) where $y < -y_\mathrm{x-point}$, b) where $y_\mathrm{x-point} < y < -y_\mathrm{x-point}$, and c) where $y > y_\mathrm{x-point}$; the general solution is

$$
n \sin(\theta) v_\parallel + n \cos(\theta) v_\perp =
\begin{cases}
C_a, &\ y < -y_\mathrm{x-point} \\
S_{p,\mathrm{core}} y + C_b, &\ \left| y \right| < y_\mathrm{x-point} \\
C_c, &\ y > y_\mathrm{x-point}
\end{cases}
$$

where $C_a$, $C_b$, and $C_c$ are constants for each region. There are thus four unknowns: $B$, $C_a$, $C_b$, and $C_c$. Requiring that $n$ and $v_\parallel$ are continuous at the boundaries between regions gives two equations, and the Bohm criterion at the lower/upper divertors gives two more equations, allowing the full system to be solved.

The Bohm criterion (accounting for modification due to the drift flow) is $v_\parallel(-y_\mathrm{divertor}) = -c_s - v_\perp / \tan(\theta)$ at the lower divertor and $v_\parallel(y_\mathrm{divertor}) = c_s - v_\perp / \tan(\theta)$ at the upper divertor [see I. Hutchinson, PoP 3, 6-7 (1996)]. Plugging these into the momentum transport equation gives

$$
\begin{align}
B &= n_{-d} \left[ \left( -c_s - \frac{v_\perp}{\tan(\theta)} \right)^2 + \frac{v_\perp}{\tan(\theta)} \left( -c_s - \frac{v_\perp}{\tan(\theta)} \right) + c_s^2 \right], \\
&= n_{-d} \left( c_s^2 + \frac{2 v_\perp c_s}{\tan(\theta)} + \frac{v_\perp^2}{\tan^2(\theta)} - \frac{v_\perp c_s}{\tan(\theta)} - \frac{v_\perp^2}{\tan^2(\theta)} + c_s^2 \right), \\
&= n_{-d} \left( 2 c_s^2 + \frac{v_\perp c_s}{\tan(\theta)} \right), \\
&= 2 c_s^2 n_{-d} \left(1 + \gamma \right),
\end{align}
$$

at the lower divertor, and

$$
\begin{align}
B &= n_{+d} \left[ \left( c_s - \frac{v_\perp}{\tan(\theta)} \right)^2 + \frac{v_\perp}{\tan(\theta)} \left( c_s - \frac{v_\perp}{\tan(\theta)} \right) + c_s^2 \right], \\
&= n_{+d} \left( c_s^2 - \frac{2 v_\perp c_s}{\tan(\theta)} + \frac{v_\perp^2}{\tan^2(\theta)} + \frac{v_\perp c_s}{\tan(\theta)} - \frac{v_\perp^2}{\tan^2(\theta)} + c_s^2 \right), \\
&= n_{+d} \left( 2 c_s^2 - \frac{v_\perp c_s}{\tan(\theta)} \right), \\
&= 2 c_s^2 n_{+d} \left(1 - \gamma \right),
\end{align}
$$

at the upper divertor, where we have defined $n_{-d} \equiv n(-y_\mathrm{divertor})$, $n_{+d} \equiv n(y_\mathrm{divertor})$, and $\gamma = v_\perp / \left( 2 c_s \tan(\theta) \right)$. We now have an important result: there is a density asymmetry caused by drifts between the upper and lower divertors, which is given by

$$
\boxed{ \frac{n_{+d}}{n_{-d}} = \frac{1 + \gamma}{1 - \gamma} }.
$$

Plugging the Bohm criterion into the particle transport equation gives

$$
\begin{align}
C_a &= n_{-d} \left[ \sin(\theta) \left( -c_s - \frac{v_\perp}{\tan(\theta)} \right) + \cos(\theta) v_\perp \right], \\
&= n_{-d} \left( -c_s \sin(\theta) - \cos(\theta) v_\perp + \cos(\theta) v_\perp \right), \\
&= -n_{-d} c_s \sin(\theta),
\end{align}
$$

at the lower divertor, and 

$$
\begin{align}
C_c &= n_{+d} \left[ \sin(\theta) \left( c_s - \frac{v_\perp}{\tan(\theta)} \right) + \cos(\theta) v_\perp \right], \\
&= n_{+d} \left( c_s \sin(\theta) - \cos(\theta) v_\perp + \cos(\theta) v_\perp \right), \\
&= n_{+d} c_s \sin(\theta),
\end{align}
$$

at the upper divertor.

The total integrated particle source into the SOL must be equal to the particle flux to the divertors, which is given by $\Gamma_\mathrm{divertor} = \left( n_{-d} + n_{+d} \right) c_s \sin(\theta)$. The total integrated particle source is $S_p^\mathrm{total} \equiv \int S_p \mathop{}\!\mathrm{d} y = 2 y_\mathrm{x-point} S_{p,\mathrm{core}}$. Setting $S_p^\mathrm{total} = \Gamma_\mathrm{divertor}$ gives

$$
\begin{align}
S_p^\mathrm{total} &= \left( n_{-d} + n_{+d} \right) c_s \sin(\theta), \\
S_p^\mathrm{total} &= n_{-d} \left( 1 + \frac{1 + \gamma}{1 - \gamma} \right) c_s \sin(\theta), \\
S_p^\mathrm{total} &= n_{-d} \left( \frac{1 - \gamma}{1 - \gamma} + \frac{1 + \gamma}{1 - \gamma} \right) c_s \sin(\theta), \\
S_p^\mathrm{total} &= \frac{2 n_{-d} c_s \sin(\theta)}{1 - \gamma}. \\
\end{align}
$$

So,

$$
\begin{align}
n_{+d} &= \frac{S_p^\mathrm{total}}{2 c_s \sin(\theta)} \left( 1 + \gamma \right), \\
n_{-d} &= \frac{S_p^\mathrm{total}}{2 c_s \sin(\theta)} \left( 1 - \gamma \right), \\
B &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)} \left( 1 - \gamma^2 \right), \\
C_a &= -\frac{S_p^\mathrm{total}}{2} \left( 1 - \gamma \right), \\
C_c &= \frac{S_p^\mathrm{total}}{2} \left( 1 + \gamma \right).
\end{align}
$$

Now we apply the interface condition that $n$ and $v_\parallel$ are continuous at the boundaries between regions. At the boundary between regions a and b (the lower X-point) this yields

$$
\begin{align}
n(-y_\mathrm{x-point}) \sin(\theta) v_\parallel(-y_\mathrm{x-point}) + n(-y_\mathrm{x-point}) \cos(\theta) v_\perp &= -n_{-d} c_s \sin(\theta), \\
n(-y_\mathrm{x-point}) \sin(\theta) v_\parallel(-y_\mathrm{x-point}) + n(-y_\mathrm{x-point}) \cos(\theta) v_\perp &= -S_{p,\mathrm{core}} y_\mathrm{x-point} + C_b.
\end{align}
$$

Subtracting these equations yields

$$
\begin{align}
C_b &= S_{p,\mathrm{core}} y_\mathrm{x-point} - n_{-d} c_s \sin(\theta), \\
&= \frac{S_p^\mathrm{total}}{2} - \frac{S_p^\mathrm{total}}{2 c_s \sin(\theta)} \left( 1 - \gamma \right) c_s \sin(\theta), \\
&= \frac{S_p^\mathrm{total}}{2} - \frac{S_p^\mathrm{total}}{2} \left( 1 - \gamma \right), \\
&= \frac{S_p^\mathrm{total}}{2} \left(1 - 1 + \gamma \right), \\
&= \frac{S_p^\mathrm{total}}{2} \gamma.
\end{align}
$$

## Equation solutions

The solved particle and momentum transport equations are thus

$$
\begin{align}
n \sin(\theta) v_\parallel + n \cos(\theta) v_\perp &=
\begin{cases}
-\left( 1 - \gamma \right) S_p^\mathrm{total} / 2, &\ y < -y_\mathrm{x-point}, \\
\left(y / y_\mathrm{x-point} + \gamma \right) S_p^\mathrm{total} / 2, &\ \left| y \right| \leq y_\mathrm{x-point}, \\
\left( 1 + \gamma \right) S_p^\mathrm{total} / 2, &\ y > y_\mathrm{x-point}, \\
\end{cases} \\
n \left( v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right) &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)} \left( 1 - \gamma^2 \right), \\
\end{align}
$$

where $S_p^\mathrm{total} = 2 y_\mathrm{x-point} S_{p,\mathrm{core}}$ and $\gamma = v_\perp / (2 c_s \tan(\theta))$. These implicit equations are nasty to solve explicitly for $n$ and $v_\parallel$, so they are numerically solved below.

The solutions can also be written in terms of normalized variables $M_\parallel = v_\parallel / c_s$, $n_* = n / n_\mathrm{max}$, and $y_* = y / y_\mathrm{x-point}$, which makes them much more compact and conditions the equations so that they can be numerically solved without ill-conditioning problems. The normalized solutions are

$$
\begin{align}
n_* \left( M_\parallel + 2 \gamma \right) &= \frac{1}{2}
\begin{cases}
-\left( 1 - \gamma \right), &\ y_* < -1, \\
\left(y_* + \gamma \right), &\ \left| y_* \right| \leq 1, \\
\left( 1 + \gamma \right), &\ y_* > 1, \\
\end{cases} \\
n_* \left( M_\parallel^2 + 2 \gamma M_\parallel + 1 \right) &= \left( 1 - \gamma^2 \right). \\
\end{align}
$$

## Insights extracted from solutions

### Upper/lower divertor density asymmetry

One key prediction of this model is the ratio of upper divertor density to lower divertor density:

$$
\boxed{ \frac{n_{+d}}{n_{-d}} = \frac{1 + \gamma}{1 - \gamma} }.
$$

### Shift of the parallel flow stagnation point

The parallel flow stagnation point is the point where $v_\parallel = 0$. When there are no drifts, this point is at the symmetry position half-way between targets ($y = 0$), which is also the location of maximum density. Drifts shift this point poloidally.

The density, $n_s$, at the parallel flow stagnation point is easily obtained from the solved momentum transport equation, giving

$$
\boxed{ n_s = \frac{S_p^\mathrm{total}}{c_s \sin(\theta)} \left( 1 - \gamma^2 \right) }.
$$

The poloidal position of the parallel flow stagnation point is now found from the solution of the particle transport equation between X-points:

$$
\begin{align}
n_s \cos(\theta) v_\perp &= \frac{S_p^\mathrm{total}}{2} \left(y_s / y_\mathrm{x-point} + \gamma \right), \\
\frac{S_p^\mathrm{total}}{c_s \sin(\theta)} \left( 1 - \gamma^2 \right) \cos(\theta) v_\perp &= \frac{S_p^\mathrm{total}}{2} \left(y_s / y_\mathrm{x-point} + \gamma \right), \\
2 \gamma S_p^\mathrm{total} \left( 1 - \gamma^2 \right) &= \frac{S_p^\mathrm{total}}{2} \left(y_s / y_\mathrm{x-point} + \gamma \right), \\
4 \gamma - 4 \gamma^3 &= \left(y_s / y_\mathrm{x-point} + \gamma \right), \\
y_s / y_\mathrm{x-point} &= -4 \gamma^3 + 3 \gamma, \\
y_s / y_\mathrm{x-point} &= \gamma \left( 3 - 4 \gamma^2 \right),
\end{align}
$$

so,

$$
\boxed{ y_s / y_\mathrm{x-point} = \gamma \left( \sqrt{3} + 2 \gamma \right) \left( \sqrt{3} - 2 \gamma \right)}.
$$

This equation implies $\left| \gamma \right|$ cannot exceed $1/2$, since for this value of the drift strength the stagnation point has moved all the way to the X-point. Since $\left| \gamma \right| < 1$, $y_s / y_\mathrm{x-point} \sim 3 \gamma$, i.e., the stagnation point shifts poloidally three times faster with increasing drift strength than the maximum density location shifts.

### Shift of the location where density is maximum

Note that the stagnation point is not the location of maximum density when drifts are present (as it is in the absence of drifts). The maximum density in this model has the same value as in the no-drift model:

$$
\boxed{ n_\mathrm{max} = \frac{S_p^\mathrm{total}}{c_s \sin(\theta)} }.
$$

This can be shown by expanding the momentum transport equation:

$$
\begin{align}
\frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ n \left( v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right) \right] &= 0, \\
\frac{\mathop{}\!\mathrm{d} n}{\mathop{}\!\mathrm{d} y} \left( v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right) + n \frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right] &= 0. \\
\end{align}
$$

Setting $\frac{\mathop{}\!\mathrm{d} n}{\mathop{}\!\mathrm{d} y} = 0$ and solving for the parallel velocity at the location of maximum density, $v_{\parallel,n_\mathrm{max}}$, gives

$$
\begin{align}
\left. \frac{\mathop{}\!\mathrm{d}}{\mathop{}\!\mathrm{d} y} \left[ v_\parallel^2 + \frac{v_\parallel v_\perp}{\tan(\theta)} + c_s^2 \right] \right|_{n_\mathrm{max}} &= 0, \\
\left( 2 v_{\parallel,n_\mathrm{max}} + \frac{v_\perp}{\tan(\theta)} \right) \left. \frac{\mathop{}\!\mathrm{d} v_\parallel}{\mathop{}\!\mathrm{d} y} \right|_{n_\mathrm{max}} &= 0. \\
\end{align}
$$

So at the location of maximum density, either the flow acceleration is zero or $v_{\parallel,n_\mathrm{max}} = -\gamma c_s$. The flow acceleration is zero beyond the X-points but is non-zero between X-points. Plugging $v_{\parallel,n_\mathrm{max}} = -\gamma c_s$ into the momentum transport equation solution to find $n_\mathrm{max}$ between X-points gives

$$
\begin{align}
n_\mathrm{max} \left( \gamma^2 c_s^2 - \frac{\gamma c_s v_\perp}{\tan(\theta)} + c_s^2 \right) &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)} \left( 1 - \gamma^2 \right), \\
n_\mathrm{max} \left( \gamma^2 c_s^2 - 2 \gamma^2 c_s^2 + c_s^2 \right) &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)} \left( 1 - \gamma^2 \right), \\
n_\mathrm{max} c_s^2 \left( 1 - \gamma^2 \right) &= \frac{c_s S_p^\mathrm{total}}{\sin(\theta)} \left( 1 - \gamma^2 \right), \\
n_\mathrm{max} &= \frac{S_p^\mathrm{total}}{c_s \sin(\theta)}. \\
\end{align}
$$

And at this point

$$
\boxed{ v_{\parallel,n_\mathrm{max}} = -\gamma c_s }.
$$

The poloidal position of this maximum density point $y_{n_\mathrm{max}}$ can be found by plugging $n_\mathrm{max}$ into the particle transport equation solution between X-points:

$$
\begin{align}
n_\mathrm{max} \left[ \sin(\theta) v_{\parallel,n_\mathrm{max}} + \cos(\theta) v_\perp \right] &= \frac{S_p^\mathrm{total}}{2} \left(y_{n_\mathrm{max}} / y_\mathrm{x-point} + \gamma \right), \\
\frac{S_p^\mathrm{total}}{c_s \sin(\theta)} \left[ -\gamma c_s \sin(\theta) + \cos(\theta) v_\perp \right] &= \frac{S_p^\mathrm{total}}{2} \left(y_{n_\mathrm{max}} / y_\mathrm{x-point} + \gamma \right), \\
-\gamma + \frac{v_\perp}{c_s \tan(\theta)} &= \frac{1}{2} \left(y_{n_\mathrm{max}} / y_\mathrm{x-point} + \gamma \right), \\
-\gamma + 2 \gamma &= \frac{1}{2} \left(y_{n_\mathrm{max}} / y_\mathrm{x-point} + \gamma \right), \\
2 \gamma &= y_{n_\mathrm{max}} / y_\mathrm{x-point} + \gamma, \\
\end{align}
$$

so,

$$
\boxed{ y_{n_\mathrm{max}} / y_\mathrm{x-point} = \gamma }.
$$

### Location where net poloidal flow is zero

At some poloidal location the net poloidal flow, i.e. the poloidal projections of the parallel flow and drift flow, will be zero. This point is highly relevant for impurity transport, since at this location impurities would not be flushed to the target but would remain in place and could build up to high density, making diffusive radial transport strong. This poloidal location could then be the an effective "leakage" channel by which impurities could escape entrainment near the targets and migrate into the confined plasma.

The net poloidal flow is $v_y = \sin(\theta) v_\parallel + \cos(\theta) v_\perp$. Setting this to zero and plugging into the solution of the particle transport equation between X-points gives

$$
\begin{align}
0 &= \frac{S_p^\mathrm{total}}{2} \left(y_\mathrm{pol,stag} / y_\mathrm{x-point} + \gamma \right), \\
\end{align}
$$

so,
$$
\boxed{ y_{\mathrm{pol,stag}} / y_\mathrm{x-point} = -\gamma }.
$$

Thus, the net poloidal flow stagnation point shifts poloidally in the *opposite* direction as the drift.

The parallel flow velocity at this point is $v_{\parallel,\mathrm{pol,stag}} = -2 c_s \gamma$ and the density at this point is $n_\mathrm{pol,stag} = n_\mathrm{max} \left( 1 - \gamma^2 \right)$.

## Routine for solving simple SOL drift model differential equations

### Input parameters

In [9]:
# Input parameters
y_divertor = 0.32 # poloidal position in meters of divertors
y_xpoint = 0.16 # poloidal position in meters of X-points
num_points = 201 # number of mesh node points
drift_velocity = 100 # meters per second
field_line_pitch_angle = 0.001 # radians
ion_mass = constants.proton_mass # kilograms
temperature = 80 # electron-volts
separatrix_density = 1e19 # particles per cubic meter, I use this as an input parameter instead of core particle source since I have more intuition on what numbers are reasonable
diffusion_coefficient = 1 # Diffusion coefficient for anomalous particle transport in meters squared per second
island_width = 0.1 # Radial island width in meters
magnetic_field = 2.5 # Magnetic field in teslas; positive for forward field, negative for reverse field

# Derived input parameters
y_norm = np.linspace(-y_divertor, y_divertor, num=num_points) / y_xpoint # Poloidal position normalized by X-point position
ion_sound_speed = np.sqrt(2 * temperature * constants.e / ion_mass)
connection_length = 2 * y_divertor / np.sin(field_line_pitch_angle) # This simple relation will underestimate real connection length because pitch angle is not actually constant (it decreases near X-point)
density_decay_length = np.sqrt(2 * diffusion_coefficient * connection_length / ion_sound_speed) # Characteristic exponential decay length of the density in meters (Stangeby textbook Eq. 4.7)
gamma = drift_velocity / (2 * ion_sound_speed * np.tan(field_line_pitch_angle))

In [3]:
gamma

0.4038811310867017

### Solve equations

In [10]:
def transport_diff_equations(density, parallel_velocity, y_norm, gamma):
    
    # Calculate the coefficients in front of the density and velocity derivative terms for the particle and momentum transport equations
    A = parallel_velocity**2 + 2*gamma*parallel_velocity + 1
    B = 2*density*(parallel_velocity + gamma)
    C = parallel_velocity + 2*gamma
    D = density

    # Define normalized particle source
    source = np.zeros_like(y_norm)
    source[(y_norm >= -1) & (y_norm <= 1)] = 1/2 # particle source is non-zero only between X-points

    # Calculate derivatives of density and parallel velocity w.r.t. poloidal position
    denom = D - B * C / A
    density_derivative = np.divide(-source * (B / A), denom, out=np.zeros_like(source), where=~np.isclose(denom, 0))
    parallel_velocity_derivative = np.divide(source, denom, out=np.zeros_like(source), where=~np.isclose(denom, 0))

    return np.array([density_derivative, parallel_velocity_derivative])

def boundary_condition_residual(density_lower_divertor, parallel_velocity_lower_divertor, density_upper_divertor, parallel_velocity_upper_divertor, gamma):
    bohm_criterion_lower_divertor = -2*gamma - 1
    bohm_criterion_upper_divertor = -2*gamma + 1
    return np.array([parallel_velocity_lower_divertor - bohm_criterion_lower_divertor, parallel_velocity_upper_divertor - bohm_criterion_upper_divertor])

# initial_guess_solution = np.stack((np.ones_like(y_norm), np.linspace(-1, 1, num=len(y_norm)))) # Guess that density is 1 everywhere and flow profile is linear with sonic flow at targets
initial_guess_solution = np.stack((np.ones_like(y_norm), np.linspace(-1 - 2*gamma, 1 - 2*gamma, num=len(y_norm)))) # Guess that density is 1 everywhere and flow profile is linear satisfying target BCs

solution = integrate.solve_bvp(
    fun=lambda x, y : transport_diff_equations(y[0, :], y[1, :], x, gamma),
    bc=lambda ya, yb : boundary_condition_residual(ya[0], ya[1], yb[0], yb[1], gamma),
    x=y_norm,
    y=initial_guess_solution,
    # max_nodes=100000,
)

solution

       message: The maximum number of mesh nodes is exceeded.
       success: False
        status: 1
             x: [-2.000e+00 -1.980e+00 ...  1.980e+00  2.000e+00]
           sol: <scipy.interpolate._interpolate.PPoly object at 0x154d82b877e0>
             p: None
             y: [[ 2.960e-01  2.960e-01 ...  6.953e-01  6.953e-01]
                 [-1.808e+00 -1.808e+00 ...  1.922e-01  1.922e-01]]
            yp: [[ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                 [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]]
 rms_residuals: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
         niter: 9

### Plot solution

In [11]:
density, parallel_velocity = solution.sol(y_norm)

fig, ax = plt.subplots()
ax.plot(y_norm, density, label='density')
ax.plot(y_norm, parallel_velocity, label='parallel_velocity')
ax.set_xlabel('y_norm')
ax.legend()
ax.set_title('Solution from numerically solved ODEs');

qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


## Routine for solving simple SOL drift model implicit solution equations

In [12]:
def transport_equations(density, parallel_velocity, y_norm, gamma):
    """Note: in these equations density is normalized by the max density and parallel velocity is normalized by the ion sound speed to improve numerical stability of the solver"""
    if y_norm < -1: # below lower X-point
        particle_transport_equation = density * (parallel_velocity + 2 * gamma) - (gamma - 1) / 2
    elif y_norm > 1: # above upper X-point
        particle_transport_equation = density * (parallel_velocity + 2 * gamma) - (gamma + 1) / 2
    else: # between X-points
        particle_transport_equation = density * (parallel_velocity + 2 * gamma) - (gamma + y_norm) / 2
    momentum_transport_equation = density * (parallel_velocity**2 + 2 * gamma * parallel_velocity + 1) - (1 - gamma**2)
    return [
        particle_transport_equation,
        momentum_transport_equation,
    ]

def solve_simple_sol_drift_model(y_norm, gamma, debug=False):
    """
    Numerically solves the coupled solutions to the 1D poloidal particle and momentum transport equations.
    y_norm is a normalized poloidal distance coordinate so that y=0 is half-way between X-points, y=-1 is at lower X-point, and y=1 is at upper X-point.
    gamma is the normalized drift velocity, given by gamma = drift_velocity / (2 * ion_sound_speed * tan(field_line_pitch_angle)).
    Note that the output density is normalized by the max value and the parallel velocity is normalized by the ion sound speed.
    """
    # Numerically solve the system of implicit equations individually at each position
    parallel_velocity = np.zeros_like(y_norm)
    density = np.zeros_like(y_norm)
    solution_found = np.zeros_like(y_norm)
    solver_message = []
    for i in range(len(y_norm)):
        (density[i], parallel_velocity[i]), _, solution_found[i], mesg  = optimize.fsolve(
            func=lambda x : transport_equations(x[0], x[1], y_norm[i], gamma),
            x0=[1, 0],
            full_output=True,
        )
        solver_message.append(mesg)
    # density[solution_found != 1] = np.nan
    # parallel_velocity[solution_found != 1] = np.nan
    if debug:
        print(solver_message)

    return density, parallel_velocity

## Use Langmuir probe measurements to estimate model input parameters

In [13]:
log_density_asymmetry = 0.5
electron_temperature = 40 # electron-volts
field_line_pitch_angle = 0.001 # radians

density_asymmetry = np.exp(log_density_asymmetry)
gamma = (density_asymmetry - 1) / (density_asymmetry + 1)
stagnation_point_shift = -4 * gamma**3 + 3 * gamma
ion_sound_speed = np.sqrt(2 * electron_temperature * constants.e / constants.proton_mass)
drift_velocity = gamma * 2 * ion_sound_speed * np.tan(field_line_pitch_angle)

print(f'density asymmetry: {density_asymmetry:.4f}')
print(f'gamma: {gamma:.3f}')
print(f'stagnation point shift: {stagnation_point_shift:.4f}')
print(f'ion sound speed: {ion_sound_speed/1000:.2f} km/s')
print(f'drift velocity: {drift_velocity:.2f} m/s')

density asymmetry: 1.6487
gamma: 0.245
stagnation point shift: 0.6760
ion sound speed: 87.54 km/s
drift velocity: 42.88 m/s


## Calculate density, parallel flow, and poloidal flow profiles

In [14]:
# Input parameters
y = np.linspace(-1, 1, num=201) # poloidal distance variable runs from -1 (lower divertor) to +1 (upper divertor)
y_xpoint = 0.75 # relative distance of x-points from upstream position to divertors
drift_velocity = -62 # meters per second
field_line_pitch_angle = 0.001 # radians
ion_mass = constants.proton_mass # kilograms
temperature = 80 # electron-volts
max_density = 1e19 # particles per cubic meter, I use this as an input parameter instead of core particle source since I have more intuition on what numbers are reasonable

# Derived parameters
y_norm = y / y_xpoint
ion_sound_speed = np.sqrt(2 * temperature * constants.e / ion_mass)
gamma = drift_velocity / (2 * ion_sound_speed * np.tan(field_line_pitch_angle))

# Run calculation
density, parallel_velocity = solve_simple_sol_drift_model(y_norm, gamma, debug=False)

# Un-normalized outputs
density = density * max_density
parallel_velocity = parallel_velocity * ion_sound_speed

# Calculate poloidal velocity
poloidal_velocity = parallel_velocity * np.sin(field_line_pitch_angle) + drift_velocity * np.cos(field_line_pitch_angle)

## Plot density and parallel flow profiles

In [15]:
plt.style.available
#plt.style.use('presentation')

['Solarize_Light2',
 '_classic_test_patch',
 '_mpl-gallery',
 '_mpl-gallery-nogrid',
 'bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'seaborn-v0_8',
 'seaborn-v0_8-bright',
 'seaborn-v0_8-colorblind',
 'seaborn-v0_8-dark',
 'seaborn-v0_8-dark-palette',
 'seaborn-v0_8-darkgrid',
 'seaborn-v0_8-deep',
 'seaborn-v0_8-muted',
 'seaborn-v0_8-notebook',
 'seaborn-v0_8-paper',
 'seaborn-v0_8-pastel',
 'seaborn-v0_8-poster',
 'seaborn-v0_8-talk',
 'seaborn-v0_8-ticks',
 'seaborn-v0_8-white',
 'seaborn-v0_8-whitegrid',
 'tableau-colorblind10']

In [16]:
fig, ax1 = plt.subplots(constrained_layout=True)
ax2 = ax1.twinx()
# ax1.set_position(pos)
# ax2.set_position(pos)
ax1.set_xticks([-1, -y_xpoint, 0, y_xpoint, 1])
ax1.set_xticklabels(['lower divertor', 'lower X-point', '0', 'upper X-point', 'upper divertor'], minor=False, rotation=60)
# ax1.set_xticklabels(['$-y_\mathrm{div}$', '$-y_\mathrm{X}$', '0', '$y_\mathrm{X}$', '$y_\mathrm{div}$'], minor=False)
# ax1.set_xticklabels(['', '', '', '', ''])
# ax1.set_xlabel('y')
# ax1.set_title(fr'Profiles for $\gamma = {gamma:.3f}$')

ax1.plot(y, parallel_velocity, linestyle='--', color='tab:blue', label=r'$v_\parallel$')
scale_factor = 1 / np.sin(field_line_pitch_angle)
ax1.plot(y, scale_factor * poloidal_velocity, linestyle=':', color='tab:blue', label=fr'${scale_factor:.0f} * v_\theta$')
# ax1.axhline(0, linestyle=':', linewidth=1.5)
ax1.set_yticks([-ion_sound_speed, 0, ion_sound_speed])
ax1.set_yticklabels(['$-c_s$', '0', '$c_s$'], minor=False)
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_ylabel(r'Velocity', color='tab:blue')
ax1.legend()
ax1.grid()

ax2.plot(y, density, color='tab:red')
ax2.set_ylim(bottom=0)
ax2.set_yticks([0, max_density])
ax2.set_yticklabels(['0', '$n_\mathrm{{max}}$'], minor=False)
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.set_ylabel('Density', color='tab:red');

fig.savefig(f'SimpleDriftModelProfiles_gamma={gamma:.3f}.pdf', dpi=300)
print(max_density)
print(min(density))
np.save("Kriete_drifts_density", density)
np.save("Kriete_drifts_parallel_velocity", parallel_velocity)
np.save("Kriete_drifts_y", y)

1e+19
3.7479685863373757e+18


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


## Calculate profiles between X-points for a scan of $\gamma$

In [11]:
# Model inputs
y_norm = np.linspace(-1, 1, num=201)
gamma = np.linspace(-0.5, 0.5, num=21)

# Initialize empty arrays to store model outputs
shape = (len(gamma), len(y_norm))
density_norm = np.zeros(shape)
parallel_mach_number = np.zeros(shape)

# Calculate profiles for each gamma value
for i in range(len(gamma)):
    density_norm[i], parallel_mach_number[i] = solve_simple_sol_drift_model(y_norm, gamma[i])

# Package results in dataset
profiles = xr.Dataset(
    data_vars={'density_norm':(('gamma', 'y_norm'), density_norm), 'parallel_mach_number':(('gamma', 'y_norm'), parallel_mach_number)},
    coords={'y_norm':y_norm, 'gamma':gamma},
)

# Test plot of dataset results
# fig, ax1 = plt.subplots()
# ax2 = ax1.twinx()
# profiles.parallel_mach_number.sel(gamma=0.25).plot(ax=ax1)
# profiles.density_norm.sel(gamma=-0.25).plot(ax=ax2)
# ax2.set_title('')

# Save dataset to file
profiles.to_netcdf('simple_sol_drift_model_profiles.nc')

In [12]:
gamma = -0.5
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
profiles.parallel_mach_number.sel(gamma=gamma).plot(ax=ax1)
profiles.density_norm.sel(gamma=gamma).plot(ax=ax2)
ax2.set_title('')

qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


Text(0.5, 1.0, '')